In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load your data
data = pd.read_csv("./aos_data/merged_phuc_data.csv")  # Replace with your file
features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ', 'IR']
X = data[features]
y = data['Label'].map({'sleep': 0, 'awake': 1})  # Convert labels to 0/1

# Normalize data (save min/max for ESP32)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Decision Tree
model = DecisionTreeClassifier(max_depth=4)  # Keep it simple for ESP32
model.fit(X_train, y_train)

# Evaluate
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")

# Export min/max values for normalization
for i, feature in enumerate(features):
    print(f"{feature}_min = {scaler.data_min_[i]};")
    print(f"{feature}_max = {scaler.data_max_[i]};")

# Manually inspect the tree (for translation to C)
from sklearn.tree import export_text
tree_rules = export_text(model, feature_names=features)
print(tree_rules)

Accuracy: 0.69
AccelX_min = 9.196129;
AccelX_max = 10.69969;
AccelY_min = -0.258574;
AccelY_max = 0.653617;
AccelZ_min = 0.998382;
AccelZ_max = 1.668759;
GyroX_min = -0.274456;
GyroX_max = 0.079406;
GyroY_min = 0.00906;
GyroY_max = 0.037305;
GyroZ_min = -0.055158;
GyroZ_max = 0.002665;
IR_min = 115848.0;
IR_max = 135484.0;
|--- GyroY <= 0.53
|   |--- IR <= 0.29
|   |   |--- IR <= 0.27
|   |   |   |--- class: 0
|   |   |--- IR >  0.27
|   |   |   |--- IR <= 0.27
|   |   |   |   |--- class: 1
|   |   |   |--- IR >  0.27
|   |   |   |   |--- class: 0
|   |--- IR >  0.29
|   |   |--- IR <= 0.79
|   |   |   |--- GyroX <= 0.63
|   |   |   |   |--- class: 1
|   |   |   |--- GyroX >  0.63
|   |   |   |   |--- class: 0
|   |   |--- IR >  0.79
|   |   |   |--- IR <= 0.84
|   |   |   |   |--- class: 0
|   |   |   |--- IR >  0.84
|   |   |   |   |--- class: 0
|--- GyroY >  0.53
|   |--- IR <= 0.36
|   |   |--- IR <= 0.29
|   |   |   |--- IR <= 0.27
|   |   |   |   |--- class: 0
|   |   |   |--- IR